In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from keras import backend as K
import matplotlib.pyplot as plt 
from tensorflow.keras.optimizers import Adam

In [ ]:
# Import data
data = pd.read_csv("../../../data/mturk_experiment_2.csv",encoding='unicode_escape')
labels = data["Formality"]
samples = data["Sentence"]

# Testing with 90% training data
train_samples, test_samples, train_labels,test_labels = train_test_split(samples, labels, test_size=0.2,random_state=5)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2,min_delta=0.001)

In [ ]:
# CNN with BERT

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3'
tfhub_handle_preprocess =   'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)


text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
net = outputs['sequence_output']
x = tf.keras.layers.Conv1D(128, 1, activation="relu")(net)
x = tf.keras.layers.MaxPooling1D(1)(x)
x = tf.keras.layers.Conv1D(128, 1, activation="relu")(x)
x = tf.keras.layers.MaxPooling1D(1)(x)
x = tf.keras.layers.Conv1D(128, 1, activation="relu")(x)
x = tf.keras.layers.GlobalMaxPooling1D()(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
preds = tf.keras.layers.Dense(units=1)(x)

cnn = tf.keras.Model(text_input, preds)
cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.summary()

In [ ]:
# LSTM with BERT

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3'
tfhub_handle_preprocess =   'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)


text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']
reshaped = tf.reshape(net,[-1, 768, 1])
x = tf.keras.layers.LSTM(4,input_shape=(1,5))(reshaped)
out = tf.keras.layers.Dense(1,activation='relu')(x)

lstm = tf.keras.Model(text_input, out)
lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.summary()

In [ ]:
# RNN with BERT

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3'
tfhub_handle_preprocess =   'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3'
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)


text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']
reshaped = tf.reshape(net,[-1, 768, 1])
x = tf.keras.layers.GRU(256,return_sequences=True)(reshaped)
x = tf.keras.layers.SimpleRNN(128)(x)
out = tf.keras.layers.Dense(1,activation='relu')(x)


rnn = tf.keras.Model(text_input, out)
rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.summary()

In [ ]:
results = pd.DataFrame()

In [ ]:
cnn.fit(x=train_samples,y=train_labels,batch_size=32,epochs=20,verbose=2,callbacks=[callback])
scores = cnn.evaluate(x=test_samples,y=test_labels)
results["CNN B32 E20"] = scores
results.T

In [ ]:
lstm.fit(x=train_samples,y=train_labels,batch_size=10,epochs=20,verbose=2,callbacks=[callback])
scores = lstm.evaluate(x=test_samples,y=test_labels)
results["LSTM B10 E20"] = scores
results.T

In [ ]:
rnn.fit(x=train_samples,y=train_labels,batch_size=10,epochs=50,verbose=2,callbacks=[callback])
scores = cnn.evaluate(x=test_samples,y=test_labels)
results["RNN B10 E50"] = scores
results.T

In [ ]:
results = results.T
results.to_csv("../../data/random_state_five/bert_cnn_rnn_lstm.csv")
results